# Tutorial 3: Reciprocity and Generalised Sea Level Forcing

In the previous tutorials, we explored how to compute sea-level fingerprints for a given surface mass load. In this tutorial, we will delve into a more fundamental physical property of the sea-level equation: **reciprocity**.

Reciprocity is a powerful concept in physics that relates the influence of a source at one point to the field measured at another. We will explore this concept and see how it leads to a more general way of thinking about the sea-level problem, where forces other than direct mass loads can be considered.

This tutorial will cover:
1.  **The Principle of Sea Level Reciprocity:** We'll explain the simple reciprocity relation for mass loads and demonstrate it with a numerical test.
2.  **Generalised Forcing:** We will introduce three new ways to force the sea-level equation: an imposed surface displacement, an imposed gravitational potential change, and a change in the Earth's angular momentum.
3.  **The Generalised Reciprocity Relation:** We will demonstrate the more complex reciprocity relation that holds true for this fully generalised problem.

## Setup and Initialization

We begin with our standard setup, creating a `FingerPrint` instance and setting its background state to the present day.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from pyslfp import FingerPrint, plot, IceModel, EarthModelParameters

# 1. Initialise the fingerprint model with a standard non-dimensionalisation
# This is good practice for ensuring numerical stability.
standard_nondim = EarthModelParameters.from_standard_non_dimensionalisation()
fp = FingerPrint(lmax=256, earth_model_parameters=standard_nondim)

# 2. Set the background state to the present day
fp.set_state_from_ice_ng(version=IceModel.ICE7G)

## 1. The Principle of Sea Level Reciprocity

In its simplest form, the reciprocity principle for sea level states:

> *The sea-level change at location **A** due to melting a unit of ice at location **B** is equal to the sea-level change at location **B** due to melting a unit of ice at location **A**.*

This elegant symmetry is a fundamental property of the linear, elastic sea-level equation. While we can't test it for single points in our gridded model, we can test an equivalent integral form. The integral form states that the work done by one load on the sea-level field generated by another is symmetric:
$$
\int_{\partial M} \zeta_2 \Delta SL_1 \, \mathrm{d}S = \int_{\partial M} \zeta_1 \Delta SL_2 \, \mathrm{d}S
$$
where $\zeta_1$ and $\zeta_2$ are two distinct surface mass loads, $\Delta SL_1$ and $\Delta SL_2$ are their respective sea-level fingerprints, and the integral is taken over the surface of the earth model, $\partial M$.

Let's test this numerically. We will create two different, arbitrary load patterns and show that this identity holds.

In [4]:
# Create two different and distinct surface mass loads
direct_load_1 = fp.northern_hemisphere_load(fraction=0.1)
direct_load_2 = fp.southern_hemisphere_load(fraction=0.2)

# Set a tight tolerance for the iterative solver to ensure an accurate test
rtol = 1e-9

# --- Calculate the sea-level response for each load ---
sea_level_change_1, _, _, _ = fp(direct_load=direct_load_1, rtol=rtol)
sea_level_change_2, _, _, _ = fp(direct_load=direct_load_2, rtol=rtol)


# --- Test the reciprocity identity ---
# Left-hand side of the equation
lhs = fp.integrate(direct_load_2 * sea_level_change_1)

# Right-hand side of the equation
rhs = fp.integrate(direct_load_1 * sea_level_change_2)

print(f"Reciprocity Test Results:")
print(f"  Left-Hand Side: {lhs:.4e}")
print(f"  Right-Hand Side: {rhs:.4e}")
print(f"  Are they close? {np.isclose(lhs, rhs, rtol=100*rtol)}")

Reciprocity Test Results:
  Left-Hand Side: -2.2997e-12
  Right-Hand Side: -2.2997e-12
  Are they close? True


The two values are equal to within the numerical precision of the solver, confirming that our model obeys this important physical principle.

## 2. Generalised Forcing in the Sea Level Equation

So far, we have only considered a `direct_load` (a surface mass change, $\zeta$) as the cause of sea-level change. However, the `FingerPrint` solver is more powerful. The `__call__` method can accept three additional types of "forcing" terms, which are useful for certain physical problems and essential for understanding the generalised reciprocity relation.

The optional arguments are:
* `displacement_load`: An externally imposed vertical displacement of the surface, $U_L$. One could imagine this representing, for example, the co-seismic displacement from a large earthquake.
* `gravitational_potential_load`: An externally imposed change to the gravitational potential, $\Phi_L$, independent of any surface mass.
* `angular_momentum_change`: An externally imposed change to the Earth's angular momentum, $\mathbf{M}$, which directly perturbs its rotation.

These generalised forces allow us to explore the full structure of the sea-level problem.

## 3. The Generalised Reciprocity Relation

When these new forcing terms are included, a more complex but equally powerful reciprocity relation emerges. This relationship is formally expressed as a **duality pairing** between a set of generalized forces, and the system's physical response. The necessary relation can be written:
$$
\int_{\partial M} \Delta SL^\dagger \zeta \, dS - \frac{1}{g} \int_{\partial M} \left[ \mathbf{t} \cdot \mathbf{u}^\dagger + \zeta_\phi^\dagger(\phi^\dagger + \psi^\dagger) \right] \, dS - \frac{1}{g} \mathbf{k} \cdot \boldsymbol{\omega}^\dagger 
    = \int_{\partial M} \Delta SL \zeta^\dagger \, dS - \frac{1}{g} \int_{\partial M} \left[ \mathbf{t}^\dagger \cdot \mathbf{u} + \zeta_\phi^\dagger(\phi + \psi) \right] \, dS - \frac{1}{g} \mathbf{k}^\dagger \cdot \boldsymbol{\omega}.

$$

Here $(\Delta SL, \mathbf{u}, \phi, \boldsymbol{\omega})$ are, respectively, the sea level change, displacement vector, gravitational potential change, 
and angular velocity change, associated with the generalised forces $(\zeta, \mathbf{t}, \zeta_{\phi}, \mathbf{k})$, while $\psi$ is the associated change 
in centrifugal potential. Terms with daggers are then a second pair of responses and associated forces. 

Note that the most general form of this relation involves all components of the displacement, but in `pyslfp` only the vertical component is currently implemented. 

This theorem is illustrated in the code below. Note in particular that the ```__call__``` method for the FingerPrint class can take in the generalised forces as additional keyword arguments. In all cases, the default values are ```None``` which corresponds to a zero-term. 

In [5]:
# --- Define two different, random sets of generalised forces ---

# A helper to create a random load
def random_load(fp: FingerPrint):
    f = np.random.uniform()
    return f * fp.northern_hemisphere_load() + (1-f) * fp.southern_hemisphere_load()

# Helper to create a random angular momentum perturbation
def random_angular_momentum(fp: FingerPrint):
    b = fp.mean_sea_floor_radius_si
    omega = fp.rotation_frequency_si
    load_lm = random_load(fp).expand(lmax_calc=2, normalization="ortho")
    # Note: The scaling here is arbitrary, just to get a reasonable magnitude
    return 1e-18 * omega * b**4 * load_lm.coeffs[:, 2, 1]

# Set of forces #1
direct_load_1 = random_load(fp)
displacement_load_1 = random_load(fp)
gravitational_potential_load_1 = random_load(fp)
angular_momentum_change_1 = random_angular_momentum(fp)

# Set of forces #2
direct_load_2 = random_load(fp)
displacement_load_2 = random_load(fp)
gravitational_potential_load_2 = random_load(fp)
angular_momentum_change_2 = random_angular_momentum(fp)

# --- Solve the generalised sea-level equation for both sets of forces ---

(
    sea_level_change_1,
    displacement_1,
    gravity_potential_change_1,
    angular_velocity_change_1,
) = fp(
    direct_load=direct_load_1,
    displacement_load=displacement_load_1,
    gravitational_potential_load=gravitational_potential_load_1,
    angular_momentum_change=angular_momentum_change_1,
    rtol=rtol,
)

(
    sea_level_change_2,
    displacement_2,
    gravity_potential_change_2,
    angular_velocity_change_2,
) = fp(
    direct_load=direct_load_2,
    displacement_load=displacement_load_2,
    gravitational_potential_load=gravitational_potential_load_2,
    angular_momentum_change=angular_momentum_change_2,
    rtol=rtol,
)


# --- Test the generalised reciprocity identity ---

g = fp.gravitational_acceleration_si

# Left-hand side: W(Forces_2, Response_1)
lhs_integrand = (
    direct_load_2 * sea_level_change_1
    - (
        g * displacement_load_2 * displacement_1
        + gravitational_potential_load_2 * gravity_potential_change_1
    ) / g
)
lhs = (
    fp.integrate(lhs_integrand)
    - np.dot(angular_momentum_change_2, angular_velocity_change_1) / g
)

# Right-hand side: W(Forces_1, Response_2)
rhs_integrand = (
    direct_load_1 * sea_level_change_2
    - (
        g * displacement_load_1 * displacement_2
        + gravitational_potential_load_1 * gravity_potential_change_2
    ) / g
)
rhs = (
    fp.integrate(rhs_integrand)
    - np.dot(angular_momentum_change_1, angular_velocity_change_2) / g
)


print(f"Generalised Reciprocity Test Results:")
print(f"  Left-Hand Side: {lhs:.4e}")
print(f"  Right-Hand Side: {rhs:.4e}")
print(f"  Are they close? {np.isclose(lhs, rhs, rtol=1000*rtol)}")

Generalised Reciprocity Test Results:
  Left-Hand Side: 7.5333e+00
  Right-Hand Side: 7.5333e+00
  Are they close? True


## Conclusion and Outlook

In this tutorial, we have confirmed that the `pyslfp` model satisfies both the simple and generalised sea-level reciprocity relations. The theory and its applications are discussed in detail in Al-Attar et al. (2023).

This is more than just a mathematical curiosity. This property:
* Serves as a powerful validation tool, confirming that our numerical implementation is physically self-consistent.
* Is the theoretical foundation for **adjoint modelling**. The generalised forces we introduced are precisely the "adjoint loads" needed to efficiently calculate the sensitivity of certain measurements (like satellite gravity) to the ice-mass change.

In a future tutorial, we will see how this reciprocity is elegantly handled by the `pygeoinf` operator formalism, and how it can be used to solve real-world inverse problems.

---

### Reference
> Al-Attar, D., Syvret, F., Crawford, O., Mitrovica, J. X., & Lloyd, A. J. (2023). Reciprocity and sensitivity kernels for sea level fingerprints. *Geophysical Journal International*, 236(1), 362–378.